In [1]:
GLOBAL_SEED = 7532

from numpy.random import seed
seed(GLOBAL_SEED)
from tensorflow import set_random_seed
set_random_seed(GLOBAL_SEED)

import numpy as np
import pandas as pd

from keras.models import load_model, Sequential
from keras.callbacks import Callback, EarlyStopping, History, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam
from keras.layers import BatchNormalization, Dense, Dropout, Conv2D, Flatten, MaxPool2D, SpatialDropout2D
from keras.preprocessing.image import ImageDataGenerator

from sklearn.utils.class_weight import compute_class_weight

Using TensorFlow backend.


Read in the data.

In [2]:
train_set = pd.read_csv('train_set_metadata_HMGD.csv')
valid_set = pd.read_csv('valid_set_metadata_HMGD.csv')
test_set = pd.read_csv('test_set_metadata_HMGD.csv')

Initialize constants.

In [3]:
INPUT_SHAPE = (320, 320, 3)

LEARNING_RATE = 0.0002
N_EPOCHS = 50
BATCH_SIZE = 32

Define the CNN model architecture.

In [4]:
def create_model(input_shape):
    model = Sequential()
    
    model.add(Conv2D(32, kernel_size=3, activation='elu',padding='same', 
                     input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(SpatialDropout2D(rate=0.25))
    model.add(Conv2D(32, kernel_size=3, activation='elu', padding='same'))
    model.add(MaxPool2D(2))
    model.add(BatchNormalization())

    model.add(Conv2D(64, kernel_size=3, activation='elu', padding='same'))
    model.add(BatchNormalization())
    model.add(SpatialDropout2D(rate=0.25))
    model.add(Conv2D(64, kernel_size=3, activation='elu', padding='same'))    
    model.add(MaxPool2D(2))
    model.add(BatchNormalization())  
    
    model.add(Conv2D(128, kernel_size=3, activation='elu', padding='same'))
    model.add(BatchNormalization())
    model.add(SpatialDropout2D(rate=0.25))
    model.add(Conv2D(128, kernel_size=3, activation='elu', padding='same'))
    model.add(MaxPool2D(2))
    model.add(BatchNormalization())
 
    model.add(Conv2D(256, kernel_size=3, activation='elu', padding='same'))
    model.add(BatchNormalization())
    model.add(SpatialDropout2D(rate=0.25))
    model.add(Conv2D(256, kernel_size=3, activation='elu', padding='same'))
    model.add(MaxPool2D(2))
    model.add(BatchNormalization())
    
    model.add(Conv2D(512, kernel_size=3, activation='elu', padding='same'))
    model.add(BatchNormalization())
    model.add(SpatialDropout2D(rate=0.25))
    model.add(Conv2D(512, kernel_size=3, activation='elu', padding='same'))
    model.add(MaxPool2D(2))
    model.add(BatchNormalization())

    model.add(Conv2D(1024, kernel_size=3, activation='elu', padding='same'))
    model.add(BatchNormalization())
    model.add(SpatialDropout2D(rate=0.25))
    model.add(Conv2D(1024, kernel_size=3, activation='elu', padding='same'))
    model.add(MaxPool2D(2))
    model.add(BatchNormalization())
    
    model.add(Flatten())
    model.add(Dropout(rate=0.5, seed=GLOBAL_SEED))
    model.add(Dense(units=512, activation='elu'))
    model.add(Dropout(rate=0.5, seed=GLOBAL_SEED))
    model.add(Dense(units=256, activation='elu'))
    model.add(Dropout(rate=0.5, seed=GLOBAL_SEED))
    model.add(Dense(1, activation='sigmoid'))
    
    return model

In [5]:
model = create_model(INPUT_SHAPE)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 320, 320, 32)      896       
_________________________________________________________________
batch_normalization_1 (Batch (None, 320, 320, 32)      128       
_________________________________________________________________
spatial_dropout2d_1 (Spatial (None, 320, 320, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 320, 320, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 160, 160, 32)      0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 160, 160, 32)      128       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 160, 160, 64)      18496     
__________

Compute the class weights to be used during model training in order to mitigate the class imbalances.

In [6]:
class_values = train_set['gender'].values
classes = np.unique(class_values)

weights = compute_class_weight(
    'balanced', 
    classes, 
    class_values
)
class_weights = dict(zip(classes, weights))
class_weights

{0: 1.0949726587842759, 1: 0.9201873755187876}

Train the model.

In [7]:
# Create and compile the model
adam = Adam(
    lr=LEARNING_RATE
)
model.compile(
    optimizer=adam, 
    loss='binary_crossentropy',
    metrics=['binary_accuracy']
)

# Initialize callbacks
checkpoint = ModelCheckpoint(
    'model_custom_big_epoch_{epoch:03d}.hdf5', 
    monitor='val_loss', 
    save_best_only=False, 
    save_weights_only=False
)
lr_reduction = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.5, 
    patience=4
)
early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=10
)    
callback_list = [checkpoint, lr_reduction, early_stopping]

# generate and augment training and validation data
data_generator = ImageDataGenerator(
    rotation_range=20, 
    width_shift_range=0.2, 
    height_shift_range=0.2, 
    horizontal_flip=True  
)
train_data_generator = data_generator.flow_from_dataframe(
    dataframe=train_set, 
    directory='imdb_crop/_all_photos/',
    x_col='photo_path',
    y_col='gender',
    target_size=(320, 320),
    class_mode='other',
    batch_size=BATCH_SIZE,
    seed=GLOBAL_SEED
)
valid_data_generator = data_generator.flow_from_dataframe(
    dataframe=valid_set, 
    directory='imdb_crop/_all_photos/',
    x_col='photo_path',
    y_col='gender',
    target_size=(320, 320),
    class_mode='other',
    batch_size=BATCH_SIZE,
    seed=GLOBAL_SEED
)

train_steps_per_epoch = train_data_generator.n//train_data_generator.batch_size
valid_steps_per_epoch = valid_data_generator.n//valid_data_generator.batch_size

#fit the model
history = model.fit_generator(
    generator=train_data_generator,
    steps_per_epoch=train_steps_per_epoch, 
    epochs=N_EPOCHS,
    callbacks=callback_list,
    validation_data=valid_data_generator,
    validation_steps=valid_steps_per_epoch,
    class_weight=class_weights,
    workers=4, 
    verbose=1
)

Found 165400 images.
Found 4176 images.
Epoch 1/50
5168/5168 [==============================] - 5105s 988ms/step - loss: 0.7460 - binary_accuracy: 0.6624 - val_loss: 0.4931 - val_binary_accuracy: 0.7712
Epoch 2/50
5168/5168 [==============================] - 5112s 989ms/step - loss: 0.4057 - binary_accuracy: 0.8231 - val_loss: 0.3572 - val_binary_accuracy: 0.8485
Epoch 3/50
5168/5168 [==============================] - 5166s 1000ms/step - loss: 0.3277 - binary_accuracy: 0.8683 - val_loss: 0.3176 - val_binary_accuracy: 0.8728
Epoch 4/50
5168/5168 [==============================] - 5267s 1s/step - loss: 0.3069 - binary_accuracy: 0.8785 - val_loss: 0.3214 - val_binary_accuracy: 0.8748
Epoch 5/50
5168/5168 [==============================] - 5245s 1s/step - loss: 0.2982 - binary_accuracy: 0.8840 - val_loss: 0.2998 - val_binary_accuracy: 0.8760
Epoch 6/50
5168/5168 [==============================] - 5195s 1s/step - loss: 0.2906 - binary_accuracy: 0.8874 - val_loss: 0.2946 - val_binary_accurac

Evaluate the model on the test set.

In [8]:
evaluation_data_generator = ImageDataGenerator().flow_from_dataframe(
    dataframe=test_set, 
    directory='imdb_crop/_all_photos/',
    x_col='photo_path',
    y_col='gender',
    target_size=(320, 320),
    class_mode='other',
    batch_size=1,
    seed=GLOBAL_SEED,
    shuffle=False
)

best_model = load_model('model_custom_big_epoch_029.hdf5')
best_model.evaluate_generator(generator=evaluation_data_generator, steps=len(test_set))

Found 7621 images.


[0.27182091733433256, 0.8933210864715917]

In [10]:
evaluation_data_generator = ImageDataGenerator().flow_from_dataframe(
    dataframe=test_set, 
    directory='imdb_crop/_all_photos/',
    x_col='photo_path',
    y_col='gender',
    target_size=(320, 320),
    class_mode='other',
    batch_size=1,
    seed=GLOBAL_SEED,
    shuffle=False
)

best_model = load_model('model_custom_big_epoch_034.hdf5')
best_model.evaluate_generator(generator=evaluation_data_generator, steps=len(test_set))

Found 7621 images.


[0.26853205023805626, 0.8947644666054324]